In [1]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
with open('data/talks.transcript.th-en.24-10-2019_11:12.json', 'r') as f:
    data = json.load(f)

In [2]:
talk_idx = []
idx = []
th_phrases = []
en_phrases = []

for i in range(len(data)):
    nb_phrases = min(len(data[i]['en']),len(data[i]['th']))
    for j in range(nb_phrases):
        talk_idx.append(i)
        th_phrases.append(data[i]['th'][j])
        en_phrases.append(data[i]['en'][j])
        if data[i]['en'][j][-1]=='.':
            idx.append(1)
        else:
            idx.append(0)

In [3]:
phrase_df = pd.DataFrame({'talk_idx':talk_idx,'en_phrase':en_phrases ,'th_phrase':th_phrases, 'idx':idx})
phrase_df.idx.sum()

136463

In [4]:
all_sentences = []
for i in range(phrase_df.talk_idx.max()):
    df = phrase_df[phrase_df.talk_idx==i]
    sentences = []
    for j,row in df.iterrows():
        sentences.append(row.th_phrase)
        if row.idx==1:
            sentences.append(' |')
    joined_sentences = ''.join(sentences)
    #remove parantheses like (audience claps)
    joined_sentences = re.sub(r'\([^)]*\)', '', joined_sentences)
    #skip if talk is nothing but parantheses
    if joined_sentences=='': continue
    #remove | at the last sentence if present
    if joined_sentences[-1]=='|': joined_sentences = joined_sentences[:-1]
    all_sentences.append(joined_sentences)

In [5]:
len(all_sentences)

1544

In [6]:
all_sentences[0]

'บนหลังอาชาแห้งกร่องดอนกิโฆเต้ พระเอกของเราบุกตะลุยสู้กับกองทัพยักษ์ |ในสายตาของเขา มันเป็นหน้าที่ของเขาที่จะปราบอสูรร้ายเหล่านี้ในนามแห่งหญิงอันเป็นที่รักของเขา ดุลสิเนอา |ทว่า การกระทำอันหาญกล้านี้ก็สูญเปล่า |เมื่อซานโซ่ ปันซ่า ผู้รับใช้ของเขาอธิบายครั้งแล้วครั้งเล่า ว่าสิ่งเหล่านี้จะเป็นยักษ์ก็หาไม่พวกมันเป็นเพียงกังหันลมเท่านั้น |ดอนกิโฆเต้ หาได้เสียความแน่วแน่แทงทวนของเขาเข้าไปยังใบพัดอย่างจัง |ด้วยพลังใจที่ไม่เคยถดถอยอัศวินผู้นั้นยืนขึ้นอย่างภาคภูมิและยิ่งเชื่อมั่นในปฏิบัติการของเขามากขึ้น |ลำดับเหตุการณ์นี้ครอบคลุมเรื่องราวส่วนใหญ่ของดอนกิโฆเต้ ที่เป็นที่รักมหากาพย์ ไร้ตรรกะ และมีชีวิตชีวาของ อลองโซ กีฆานาผู้กลายเป็น ดอนกิโฆเต้ แห่งลามันช่าผู้ซุ่มซ่ามแต่กล้าหาญหรือที่รู้จักกันในนาม ขุนนางต่ำศักดิ์นักฝัน |แต่เดิมวรรณกรรมนี้มีสองเล่มบรรยายเรื่องราวของดอนกิโฆเต้ในขณะที่เขาเดินทางผ่านตอนกลาง และตอนเหนือของสเปนเพื่อต่อสู้กับบรรดาปิศาจร้าย |แม้จินตนาการในเรื่อง ดอนกิโฆเต้ จะสูงล้ำเหนือเมฆผู้ประพันธ์ มิเกล เด เซร์บันเตสก็ไม่เคยนึกฝันว่าหนังสือของเขาจะกลายเป็นนิยายที่ขายดีที่สุดตลอดกาล 

In [7]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'B' if j==0 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [8]:
len(all_tuples)

1544

In [25]:
def extract_features(doc, window=2, max_n_gram = 3):
    doc_features = []
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]
    doc_pos = [p for (w,p) in pos_tag(doc)] 
    for i in range(window, len(doc)-window):
        word_features = ['bias']
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                pos_ =f'{"|".join(doc_pos[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
                word_features += [f'pos_{feature_position}={pos_}']
        doc_features.append(word_features)
    return doc_features

In [27]:
extract_features(word_tokenize('ฉันชอบกินมะนาว'), window=2, max_n_gram = 3)

[['bias',
  'word_1_-2_-1=xxpad',
  'pos_1_-2_-1=NCMN',
  'word_1_-1_0=xxpad',
  'pos_1_-1_0=NCMN',
  'word_1_0_1=ฉัน',
  'pos_1_0_1=PPRS',
  'word_1_1_2=ชอบ',
  'pos_1_1_2=VACT',
  'word_1_2_3=กิน',
  'pos_1_2_3=VACT',
  'word_2_-2_0=xxpad|xxpad',
  'pos_2_-2_0=NCMN|NCMN',
  'word_2_-1_1=xxpad|ฉัน',
  'pos_2_-1_1=NCMN|PPRS',
  'word_2_0_2=ฉัน|ชอบ',
  'pos_2_0_2=PPRS|VACT',
  'word_2_1_3=ชอบ|กิน',
  'pos_2_1_3=VACT|VACT',
  'word_3_-2_1=xxpad|xxpad|ฉัน',
  'pos_3_-2_1=NCMN|NCMN|PPRS',
  'word_3_-1_2=xxpad|ฉัน|ชอบ',
  'pos_3_-1_2=NCMN|PPRS|VACT',
  'word_3_0_3=ฉัน|ชอบ|กิน',
  'pos_3_0_3=PPRS|VACT|VACT'],
 ['bias',
  'word_1_-2_-1=xxpad',
  'pos_1_-2_-1=NCMN',
  'word_1_-1_0=ฉัน',
  'pos_1_-1_0=PPRS',
  'word_1_0_1=ชอบ',
  'pos_1_0_1=VACT',
  'word_1_1_2=กิน',
  'pos_1_1_2=VACT',
  'word_1_2_3=มะนาว',
  'pos_1_2_3=NCMN',
  'word_2_-2_0=xxpad|ฉัน',
  'pos_2_-2_0=NCMN|PPRS',
  'word_2_-1_1=ฉัน|ชอบ',
  'pos_2_-1_1=PPRS|VACT',
  'word_2_0_2=ชอบ|กิน',
  'pos_2_0_2=VACT|VACT',
  'word_2_1_3=กิ

In [ ]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_, window=3, max_n_gram = 7) for x_ in x_pre]

In [29]:
len(x),len(y)

NameError: name 'x' is not defined

In [ ]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [ ]:
x_train[0][:5]

In [ ]:
# Train model
trainer = pycrfsuite.Trainer(verbose=True)
#trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
  trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1,
    'c2': 1e-1,
    'max_iterations': 200,
    'feature.possible_transitions': True,
})

trainer.train('sentenceseg-crf.model')

In [ ]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('sentenceseg-crf.model')
y_pred = [tagger.tag(xseq) for xseq in x_test]

In [ ]:
# Evaluate
labels = {'B': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["B", "I"]))

In [ ]:
def sentence_tokenize(s):
    toks = word_tokenize(s)
    feat = extract_features(toks)
    labs = tagger.tag(feat)
    sentences = []
    sentence = ''
    for i, w in enumerate(toks):
        if labs[i] == 'B':
            if sentence:
                sentences.append(sentence)
            sentence = ''
        sentence = sentence + w
    if sentence:
        sentences.append(sentence)
    return sentences

In [ ]:
s = 'เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ เธอสามารถตัดสินใจด้วยตัวเอง แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน'

In [ ]:
sentence_tokenize(s)